# Computing ice sheet related sea level change from a CISM simulation

Global sea level varies as a function of primarily 3 component changes:
- Ocean thermal expension
- Glacier
- Ice sheets

**Note 1:** Here we are talking about **global** and not local changes. Local impact, on different time scales, needs to include the effect of tides and winds among other parameters.\
**Note 2:** In our case, we will look at the sea level changes from the Greenland ice sheet only, as that is the only actively evolving ice sheet in our simulation.\
**Note 3:** Articles have been published (e.g. [Goelzer et al. (2020)](https://tc.copernicus.org/articles/14/833/2020/)) arguing that the computation of sea level change from stand alone ice sheet model output needs to correct for other factors (e.g. bedrock elevation changes, density correction due to the small difference between ice and fresh water density). In this exercise, we will only be accounting for the changes in total ice volume above floatation. The other correcting factors account for about 10% of the total sea level change magnitude (in general). 

<div class="alert alert-info">
<strong>Step 1: Understanding ice sheet sea level contribution </strong><br><br>


Only looking at the change of mass of an ice sheet is not enough to compute sea level contribution. In fact, any ice already in the ocean (including the bits still attached to the ice sheet) is already displacing sea water, hence contributing to sea level. What we need to compute is the change of mass that is not displacing sea water, or in other words, the change of mass above floatation. <br><br> 
  
    
<strong>Step 2: Knowing the conversion </strong><br>

Because ice, fresh water, and ocean water have different densities, it takes about **360 Gt** of ice to raise the mean global sea level (MGSL) by **1 mm**. 
A good description on how this number of 360 Gt was obtained can be found here:\
[Calculating glacier ice volumes and sea level equivalents](https://www.antarcticglaciers.org/wp-content/plugins/antarcticglaciers-pdf/download.php?p=7657#:~:text=In%20the%20same%20way%2C%201,sea%20levels%20by%201%20mm.)
    
<br>
<strong>Step 3: Compute the contribution of the GrIS to sea level change from this experiment. </strong><br><br>

</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>

    
**(1) Compute the change in mass above flotation**

Luckily, this scalar is part of the CISM history output variables and is called **imass_above_flotation**
    
**(2) Create a time series for mass above flotation**
    
You can follow similar steps as done previously for ice mass.

**(3) Load the time series data and use the conversion number above to finalize your computation**

**(4) Plot your results**
    

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
    

<font face="Courier" color='purple'><strong>Create a time series for imass_above_flotation scalars in a single file</strong></font>:

```
ncrcat -v imass_above_flotation T_GrIS_SSP585_2015_2100.cism.h.*.nc mass_above_flotation.nc   
```

<font face="Courier" color='purple'><strong>Copy and execute the text bellow in a new jupyter window</strong></font>

```
#  Defining the conversion constant
mm_equiv = 360.e12    # converting Gt to kg  

# Reading in the mass above flotation data
User = 'username'
path_to_file = '/glade/scratch/' + User + '/archive/T_GrIS_SSP585_2015_2100/glc/hist/'
file_mass_above_flotation = path_to_file + "mass_above_flotation.nc"
ncfile = Dataset(file_mass_above_flotation,'r')
mass_af = ncfile.variables["imass_above_flotation"][:]
time_maf = ncfile.variables["time"][:]
ncfile.close()


# Computing the overal sea level contribution from the GrIS during this experiment
slc = -(mass_af[:] - mass_af[0])/mm_equiv
print('The GrIS contribution of global mean sea level is ',slc[-1],'mm')


# Plotting the time series of see level change
sizefontx = 13
sizefonty = 13
sizefonttitle = 13
color = 'black'
line = '-'


timemin = 2015
timemax = 2101

plt.figure(figsize=(7,5))

plt.plot(time_maf, slc, line, ms=3, mfc=color, color=color)
plt.xlim([timemin, timemax])
plt.xlabel('Time (yr)', multialignment='center',fontsize=sizefontx)
plt.ylabel('Sea level change (mm)', multialignment='center',fontsize=sizefonty)
plt.title('GrIS contribution to sea level change',fontsize=sizefonttitle)


```

</details>
</div>



<div class="alert alert-info">
<strong>Food for thought </strong><br><br>
    

(1) Based on the sea level change figure, the Greenland ice sheet has been accumulating more ice than it has lost leading to a sea level sink. What do you think about these results?

(2) Do you have any suspicions about the forcing used in this experiment? If so, what are they?
    
(3) What else could you be suspicious about?

(4) What would you do to validate these results?


<strong>Note:</strong><br> these questions are quite challenging. Contact Gunter Leguy (gunterl@ucar.edu) for further discussion.
    
</div>


<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for some answers</font></summary><br>
    

<font face="Courier" color='purple'><strong>Question 1</strong></font>
The current observations show a positive Greenland ice sheet (GrIS) contribution to sea level change of about 1 mm per year over the past decade. For this reason, it is surprising to see it contribute negatively in this set of experiments especially under a strong warming scenario and these results are highly suspicious. 

<font face="Courier" color='purple'><strong>Question 2</strong></font>
Looking at the SMB forcing, the values are negative along the coasts and positive in the interior of the ice sheet (especially in the south). Looking at the CISM thickness difference plot between the end and beginning of the simulation, the ice thickness evolution correlates closely with the changes in SMB. It is difficult (without further analyzes) to draw any conclusions about the forcing. One can always suspect whether we set the experiment properly. Also, this experiment has not yet been analyzed and one can suspect that the 2 deg version of the CESM model could lead to unexpected climate simulations. 
    
<font face="Courier" color='purple'><strong>Question 3</strong></font>
In this exercise, we have not talked about how the GrIS was initialized and then used in CESM. Initialization is one of the greatest uncertainties in ice sheet modeling. Here, the model was spun-up using the SMB and air temperature of the RACMO2.3 regional climate model. This way we could invert for the basal physics using observed ice thickness and bed topography. However, there are some biases between RACMO results and the CESM simulated SMB and air temperature which will impact the ice sheet evolution. Ideally, a simulation would correct for these biases which is not the case here.     
    
<font face="Courier" color='purple'><strong>Question 4</strong></font>
At least a couple of steps should be considered to validate these results:
1. Look at the biases between the observed fields used to initialize CISM and compare them to those in CESM at the beginning of the simulation.
2. Compare the forcing of the FV2 (2 deg model) output to the FV1 (1 deg model) output and look for potential biases due to the CESM model resolution. 

</details>
</div>

